In [1]:
# load class of structure
from graph_tree_module import node, sim, graph_tree


In [2]:
# load pre construct data
import pickle

# Open the file in binary read mode and load the object
with open("graph_tree.pkl", "rb") as file:
    gt = pickle.load(file)

print(gt)
print(gt.root.down_lv[0].center)

[0.44017957 0.51780136 0.65391656 0.44026177 0.75100433 1.
 0.05753974 0.25911434 0.13463644 0.18605759 0.47602233 0.48493261
 0.13730071 0.72769953]


In [3]:
gt.attr_name

['Popularity',
 'danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'duration_in min/ms',
 'time_signature']

In [4]:
sliders_data = [{"id": data.attr_name[i], "min": data.X_maxmin[1][i], "max": data.X_maxmin[0][i], "value": data.root.center[i], "type": "f"} for i in range(len(data.attr_name))]
print(sliders_data)

In [5]:
from flask import Flask, render_template, request, jsonify

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

# slider ####################################################################
@app.route('/sliders')
def sliders(data):
    return jsonify(sliders_data)

@app.route('/submit', methods=['POST'])
def submit_slider_values():
    data = request.json
    print('Received slider values:', data)
    # Process the data as needed
    return jsonify({'status': 'success', 'received': data})
# slider end ####################################################################

@app.route('/graph-data')
def data():
    nodes = [
        {"id": "center", "x": 300, "y": 200, "info": "Center Node", "color": "red"},
        {"id": "left", "x": 150, "y": 200, "info": "Left Node", "color": "green"},
        {"id": "top", "x": 300, "y": 100, "info": "Top Node", "color": "pink"},
        {"id": "right1", "x": 450, "y": 250, "info": "Right Node 1", "color": "blue"},
        {"id": "right2", "x": 550, "y": 200, "info": "Right Node 2", "color": "blue"},
        {"id": "bottom1", "x": 300, "y": 300, "info": "Bottom Node 1", "color": "pink"},
        {"id": "bottom2", "x": 250, "y": 350, "info": "Bottom Node 2", "color": "pink"},
        {"id": "bottom3", "x": 350, "y": 350, "info": "Bottom Node 3", "color": "pink"}
    ]
    return jsonify(nodes)

@app.route('/tree-data')
def tree_data():
    tree = {
        "name": "Root",
        "children": [
            {
                "name": "Child 1",
                "children": [
                    {"name": "Grandchild 1"},
                    {"name": "Grandchild 2"}
                ]
            },
            {
                "name": "Child 2",
                "children": [
                    {"name": "Grandchild 3"},
                    {"name": "Grandchild 4"}
                ]
            }
        ]
    }
    return jsonify(tree)

if __name__ == '__main__':
    app.run(port=8123)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8123
Press CTRL+C to quit
127.0.0.1 - - [30/Aug/2024 00:13:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Aug/2024 00:13:25] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [30/Aug/2024 00:13:25] "GET /static/scripts.js HTTP/1.1" 304 -
127.0.0.1 - - [30/Aug/2024 00:13:25] "GET /graph-data HTTP/1.1" 200 -
[2024-08-30 00:13:25,429] ERROR in app: Exception on /sliders [GET]
Traceback (most recent call last):
  File "C:\Users\lst74\AppData\Roaming\Python\Python312\site-packages\flask\app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\lst74\AppData\Roaming\Python\Python312\site-packages\flask\app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\lst74\AppData\Roaming\Python\Python312\site-packages\flask\app.py", line 880, in full_dispatch_request
    rv = self.dispatch_request()
         